In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
file_path = 'Nat_Gas.csv'  # Replace this with the path to your CSV file
data = pd.read_csv(file_path)

# Convert Dates column to datetime format for easier manipulation
data['Dates'] = pd.to_datetime(data['Dates'], format='%m/%d/%y')

# Define the pricing function
def price_gas_contract(injection_dates, withdrawal_dates, injection_rates, withdrawal_rates, max_storage, storage_cost, price_data):
    """
    Calculate the value of the gas storage contract.

    Parameters:
    - injection_dates: List of dates to inject gas (datetime objects).
    - withdrawal_dates: List of dates to withdraw gas (datetime objects).
    - injection_rates: List of volumes of gas to inject on respective injection dates (units).
    - withdrawal_rates: List of volumes of gas to withdraw on respective withdrawal dates (units).
    - max_storage: Maximum storage capacity (units).
    - storage_cost: Cost of storing gas per unit volume per day (units of currency).
    - price_data: DataFrame containing dates and gas prices.

    Returns:
    - Total value of the contract (units of currency).
    """
    # Initialize variables
    storage_volume = 0
    total_value = 0

    # Merge injection and withdrawal data into a unified time series
    price_data = price_data.set_index('Dates')
    
    # Step 1: Process Injection
    for date, rate in zip(injection_dates, injection_rates):
        if date not in price_data.index:
            raise ValueError(f"Price data is missing for the injection date {date}.")
        # Calculate purchase cost
        price = price_data.loc[date, 'Prices']
        total_value -= rate * price  # Subtract cost from value (buying gas)
        
        # Update storage volume
        storage_volume += rate
        if storage_volume > max_storage:
            raise ValueError("Storage capacity exceeded during injection.")

    # Step 2: Process Withdrawal
    for date, rate in zip(withdrawal_dates, withdrawal_rates):
        if date not in price_data.index:
            raise ValueError(f"Price data is missing for the withdrawal date {date}.")
        # Calculate sale revenue
        price = price_data.loc[date, 'Prices']
        total_value += rate * price  # Add revenue to value (selling gas)
        
        # Update storage volume
        storage_volume -= rate
        if storage_volume < 0:
            raise ValueError("Insufficient gas in storage for withdrawal.")

    # Step 3: Calculate storage costs
    # Assume storage cost applies for every day gas is in storage
    total_days = (price_data.index[-1] - price_data.index[0]).days
    average_storage = (sum(injection_rates) - sum(withdrawal_rates)) / 2  # Approximation
    total_storage_cost = average_storage * storage_cost * total_days

    total_value -= total_storage_cost  # Subtract storage costs

    return total_value

# Test the function with sample inputs
injection_dates = [datetime(2020, 10, 31), datetime(2020, 11, 30)]
withdrawal_dates = [datetime(2020, 12, 31), datetime(2021, 1, 31)]
injection_rates = [500, 300]
withdrawal_rates = [400, 200]
max_storage = 1000  # Maximum capacity
storage_cost = 0.05  # Cost per unit volume per day

# Run the function with the provided data
contract_value = price_gas_contract(
    injection_dates,
    withdrawal_dates,
    injection_rates,
    withdrawal_rates,
    max_storage,
    storage_cost,
    data
)

print("The contract value is:", contract_value)

FileNotFoundError: [Errno 2] No such file or directory: 'Nat_Gas.csv'